In [17]:
import json

# Read the json in the path "C:\Users\jiaha\Documents\Universidad\SBC\SBC-MENU\PART2\Bases_Conocimientos\ontologia_ingredientes_cultura.json"
with open(r"C:\Users\jiaha\Documents\Universidad\SBC\SBC-MENU\PART2\Otras_Bases\cbr_dataset_estructurado.json", 'r', encoding='utf-8') as f:
    data = json.load(f)

# Display the data
dict_tecnics = dict()
for case in data['menus']:
    for key in ["starter","main","dessert"]:
        for step in case['courses'][key]['structured_instructions']:
            action = step.get('preparation_action')
            if action:
                dict_tecnics[action] = dict_tecnics.get(action, 0) + 1
print("Numero de tecnicas:", len(dict_tecnics))
# Print all techniques sorted by value
for tecnic, count in sorted(dict_tecnics.items(), key=lambda item: item[1], reverse=True):
    print(f"{tecnic}: {count}")


Numero de tecnicas: 113
add: 41
remove: 17
place: 15
serve: 14
mix: 12
combine: 12
cook: 11
stir: 11
cut: 11
bake: 10
simmer: 9
heat: 9
bring: 8
chop: 7
let: 7
saut: 7
cover: 7
drain: 7
refrigerate: 7
preheat: 6
spread: 6
pour: 6
toss: 6
taste: 5
slice: 4
make: 4
drizzle: 4
set aside: 4
soak: 4
boil: 4
set: 4
cool: 4
transfer: 4
sprinkle: 4
allow: 3
peel: 3
reduce: 3
garnish: 3
fold: 3
pulse: 3
beat: 3
prepare: 3
spray: 2
roast: 2
blend: 2
line: 2
rinse: 2
lay: 2
brush: 2
spoon: 2
open: 2
reserve: 2
scoop: 2
shake: 2
puree: 2
squeeze: 2
score: 2
let sit: 2
dip: 2
whip: 2
seed: 1
flip: 1
turn off and let cool: 1
strain and pour: 1
get ready: 1
have: 1
re-heat: 1
divide: 1
top: 1
baste: 1
coat: 1
tear: 1
discard: 1
press: 1
let set: 1
crumble: 1
cut up: 1
mix in: 1
toss in: 1
dice: 1
core: 1
check: 1
let cool: 1
leave: 1
char: 1
keep: 1
gather: 1
warm: 1
turn: 1
test: 1
reduce heat: 1
season: 1
saute: 1
whisk: 1
grease: 1
invert: 1
take: 1
start: 1
roll: 1
store: 1
grease and flour: 1
ad

In [39]:
with open(r"C:\Users\jiaha\Documents\Universidad\SBC\SBC-MENU\PART2\Bases_Conocimientos\ontologia_ingredientes_cultura.json", 'r', encoding='utf-8') as f:
    data = json.load(f)
list_culturas = list(key for key in data["ontology_tree"].keys() if key not in ["Global/Common", "Unspecified"])
list_culturas

['Latin American',
 'French/Western European',
 'Mediterranean',
 'South Asian',
 'Chinese',
 'American',
 'Italian',
 'East Asian (General)',
 'Japanese',
 'Middle Eastern/North African',
 'Korean']

In [ ]:
import json
import os
import time
import requests
import re
from time import sleep

# -------------------------------------------------
# CONFIGURATION
# -------------------------------------------------
OUTPUT_JSON = r'C:\Users\jiaha\Documents\Universidad\SBC\SBC-MENU\PART2\Bases_Conocimientos\ontologia_cultura_tecnicas.json'
OUTPUT_JSON_SUBS = r'C:\Users\jiaha\Documents\Universidad\SBC\SBC-MENU\PART2\Bases_Conocimientos\ontologia_tecnica_sustituciones.json'

HF_API_KEY = os.environ.get("HUGGINGFACE_API_KEY")
HF_MODEL_NAME = "meta-llama/Llama-3.1-8B-Instruct"
HF_CHAT_URL = "https://router.huggingface.co/v1/chat/completions"

HEADERS = {
    "Authorization": f"Bearer {HF_API_KEY}",
    "Content-Type": "application/json"
}

REQUEST_DELAY = 2  # seconds between API calls

# -------------------------------------------------
# UTILS
# -------------------------------------------------
def clean_llm_json(text):
    """Remove markdown and backticks from LLM response"""
    text = re.sub(r"^```json", "", text.strip(), flags=re.IGNORECASE)
    text = re.sub(r"^```", "", text.strip(), flags=re.IGNORECASE)
    text = re.sub(r"```$", "", text.strip(), flags=re.IGNORECASE)
    return text.strip()

def parse_llm_response(content):
    """Convert LLM response to JSON"""
    text = clean_llm_json(content)
    try:
        return json.loads(text)
    except Exception as e:
        return {"error": "json_parse_failed", "details": str(e), "raw": text}
def remap_json(input_json):
    """Values become keys, keys become values (list)"""
    output_json = dict()
    for key, values in input_json.items():
        for value in values:
            if value not in output_json:
                output_json[value] = []
            output_json[value].append(key)
    return output_json

# -------------------------------------------------
# PROMPTS
# -------------------------------------------------
def create_culture_tecnic_prompt(cultures_list, techniques_list):
    """Prompt for culture -> representative techniques (with optional extras)"""
    cultures_str = ", ".join(cultures_list)
    techniques_str = ", ".join(techniques_list)
    prompt = f"""
You are a culinary knowledge expert. Create a comprehensive knowledge base that maps cooking cultures to their representative cooking techniques.

Available cultures: {cultures_str}

Available techniques: {techniques_str}

For each tecnique:
- Consider if it is strongly associated with any of the listed cultures. If so, include it in the json with 
and list of cultures it belongs to, else omit it.
- A technique can belong to multiple cultures. At most, list 5 cultures per technique.
- Be consistent with culture names as provided in the available cultures list.
Rules:
- You can add extra techniques not in the available techniques list if they are strongly representative of a culture.
- Every culture must have at least 10 techniques associated with it.
- Return ONLY a valid JSON object with this structure:
{{
  "Tecnique_name": ["culture1", "culture2", "culture3", ...],
  ...
}}

Do NOT include explanations, markdown, or additional text. Return ONLY the JSON object.
"""
    return prompt

def create_tecnic_substitution_prompt(techniques_list):
    """Prompt for technique -> substitute techniques"""
    techniques_str = ", ".join(techniques_list)
    prompt = f"""
You are a culinary expert. For each cooking technique, propose close substitutes from the provided list.

Available techniques: {techniques_str}

Rules:
- For each technique, return 3-6 alternative techniques that could replace it in a recipe.
- Use only techniques from the available list (no new ones).
- Do NOT include the original technique in its own substitution list.
- Favor substitutions that preserve texture/result as much as possible.

Return ONLY a valid JSON object with this structure:
{{
  "technique": ["substitute1", "substitute2", ...],
  ...
}}

Do NOT include explanations, markdown, or additional text. Return ONLY the JSON object.
"""
    return prompt

# -------------------------------------------------
# LLM CALL
# -------------------------------------------------
def generate_knowledge_base(prompt, output_path, max_retries=3):
    """
    Generic LLM call with retries and JSON save.
    """
    if not HF_API_KEY:
        return {"error": "HUGGINGFACE_API_KEY not set"}

    payload = {
        "model": HF_MODEL_NAME,
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.3,
        "max_new_tokens": 2000
    }

    # Retry logic with exponential backoff
    for attempt in range(max_retries):
        try:
            print(f"  📡 Calling LLM (attempt {attempt + 1}/{max_retries})...", end="", flush=True)
            response = requests.post(HF_CHAT_URL, headers=HEADERS, json=payload, timeout=120)
            
            if response.status_code == 200:
                print(" ✓")
                result = response.json()
                content = result["choices"][0]["message"]["content"]
                kb = parse_llm_response(content)
                kb = remap_json(kb)
                
                # Save to JSON
                print(f"  💾 Saving to {output_path}")
                os.makedirs(os.path.dirname(output_path), exist_ok=True)
                with open(output_path, 'w', encoding='utf-8') as f:
                    json.dump(kb, f, ensure_ascii=False, indent=2)
                
                print(f"  ✅ Saved successfully")
                return kb
                
            elif response.status_code in [502, 503, 504]:
                # Server errors - retry with backoff
                if attempt < max_retries - 1:
                    wait_time = 2 ** attempt
                    print(f" ⚠️  HTTP {response.status_code}, retrying in {wait_time}s...")
                    sleep(wait_time)
                else:
                    print(f" ✗")
                    return {"error": f"HTTP {response.status_code}", "details": response.text[:500]}
            else:
                # Other errors
                print(f" ✗")
                return {"error": f"HTTP {response.status_code}", "details": response.text[:500]}
                
        except requests.Timeout:
            if attempt < max_retries - 1:
                wait_time = 2 ** attempt
                print(f" ⚠️  Timeout, retrying in {wait_time}s...")
                sleep(wait_time)
            else:
                print(f" ✗")
                return {"error": "llm_call_failed", "details": "Request timeout after all retries"}
                
        except Exception as e:
            if attempt < max_retries - 1:
                wait_time = 2 ** attempt
                print(f" ⚠️  Error ({str(e)[:50]}), retrying in {wait_time}s...")
                sleep(wait_time)
            else:
                print(f" ✗")
                return {"error": "llm_call_failed", "details": str(e)[:500]}

In [43]:
# Example usage
if HF_API_KEY:
    # Get cultures from the ontology
    culturas_lista = list_culturas
    
    # Get techniques from the first cell
    tecnicas_lista = sorted(dict_tecnics.keys())
    
    print(f"🔬 Creating culture-technique KB with {len(culturas_lista)} cultures and {len(tecnicas_lista)} techniques...")
    prompt_cult = create_culture_tecnic_prompt(culturas_lista, tecnicas_lista)
    kb_cult = generate_knowledge_base(prompt_cult, OUTPUT_JSON)

    if "error" not in kb_cult:
        print(f"\n✅ Culture-technique KB saved: {OUTPUT_JSON}")
        # Build technique set from the generated KB (deduplicated)
        all_tecnicas_from_kb = sorted({tech for techs in kb_cult.values() for tech in techs if isinstance(techs, list)})
        print(f"🔁 Creating technique-substitution KB for {len(all_tecnicas_from_kb)} techniques...")
        prompt_subs = create_tecnic_substitution_prompt(all_tecnicas_from_kb)
        kb_subs = generate_knowledge_base(prompt_subs, OUTPUT_JSON_SUBS)

        if "error" not in kb_subs:
            print(f"\n✅ Technique-substitution KB saved: {OUTPUT_JSON_SUBS}")
            print(f"   Total techniques with substitutes: {len(kb_subs)}")
        else:
            print(f"❌ Error generating substitutions: {kb_subs}")
    else:
        print(f"❌ Error generating culture-technique KB: {kb_cult}")
else:
    print("❌ HUGGINGFACE_API_KEY not set")

🔬 Creating culture-technique KB with 11 cultures and 113 techniques...
  📡 Calling LLM (attempt 1/3)... ✓
  💾 Saving to C:\Users\jiaha\Documents\Universidad\SBC\SBC-MENU\PART2\Bases_Conocimientos\ontologia_cultura_tecnicas.json
  ✅ Saved successfully

✅ Culture-technique KB saved: C:\Users\jiaha\Documents\Universidad\SBC\SBC-MENU\PART2\Bases_Conocimientos\ontologia_cultura_tecnicas.json
🔁 Creating technique-substitution KB for 112 techniques...
  📡 Calling LLM (attempt 1/3)... ✓
  💾 Saving to C:\Users\jiaha\Documents\Universidad\SBC\SBC-MENU\PART2\Bases_Conocimientos\ontologia_tecnica_sustituciones.json
  ✅ Saved successfully

✅ Technique-substitution KB saved: C:\Users\jiaha\Documents\Universidad\SBC\SBC-MENU\PART2\Bases_Conocimientos\ontologia_tecnica_sustituciones.json
   Total techniques with substitutes: 93


In [ ]:
# Read the json in the path OuTPUT_JSON_SUBS and OUTPUT_JSON and display them
with open(OUTPUT_JSON_SUBS, 'r', encoding='utf-8') as f:
    x = json.load(f)
data_sub = dict()
for key, value in x.items():
    data_sub[key] = set(value)
with open(OUTPUT_JSON, 'r', encoding='utf-8') as f:
    x = json.load(f)
data_cult = dict()
for key, value in x.items():
    data_cult[key] = set(value)

In [ ]:
# Quiero crear una matriz de distancia entre culturas basado en las tecnicas de cocina que tiene uno pero no el otro.
culturas = list(data_cult.keys())
import numpy as np
dist_matrix = np.zeros((len(culturas), len(culturas)), dtype=int)
for i in range(len(culturas)):
    for j in range(i+1, len(culturas)):
        cult1 = culturas[i]
        cult2 = culturas[j]
        tecnicas_cult1 = data_cult[cult1]
        tecnicas_cult2 = data_cult[cult2]
        """common_tecnicas = tecnicas_cult1.intersection(tecnicas_cult2)
        if len(common_tecnicas) == min(len(tecnicas_cult1), len(tecnicas_cult2)):
            dist_matrix[i, j] = -1
            dist_matrix[j, i] = -1
            continue"""
        # Calcular la distancia como el numero de tecnicas que tiene cult1 pero no cult2
        dist = len(tecnicas_cult1 - tecnicas_cult2)
        dist_matrix[i, j] = dist
        # Callculamos el numero de tecnicas que tiene cult2 pero no cult1
        dist = len(tecnicas_cult2 - tecnicas_cult1)
        dist_matrix[j, i] = dist

# Display the distance matrix
import pandas as pd 
df_dist = pd.DataFrame(dist_matrix, index=culturas, columns=culturas)
df_dist
# Result explanation: The distance matrix shows the number of unique cooking techniques that one culture has compared to another. 
# A value of -1 indicates that one culture's techniques are a subset of the other's.
# For example, if df_dist.loc['Italian', 'French'] = 5, it means Italian cuisine has 5 cooking techniques that French cuisine does not have.
# And df_dist.loc['French', 'Italian'] = 3 means French cuisine has 3 cooking techniques that Italian cuisine does not have.
    

,Latin American,Mediterranean,South Asian,Chinese,Italian,French/Western European,Middle Eastern/North African,East Asian,Japanese
Latin American,0,0,0,0,0,30,30,30,30
Mediterranean,82,0,82,82,78,30,34,30,30
South Asian,0,0,0,0,0,30,30,30,30
Chinese,0,0,0,0,0,30,30,30,30
Italian,4,0,4,4,0,30,34,30,30
French/Western European,82,0,82,82,78,0,4,0,0
Middle Eastern/North African,78,0,78,78,78,0,0,0,0
East Asian,82,0,82,82,78,0,4,0,0
Japanese,82,0,82,82,78,0,4,0,0


In [67]:
for key, value in data_cult.items():
    print(f"{key}: {len(value)} techniques")

Latin American: 30 techniques
Mediterranean: 112 techniques
South Asian: 30 techniques
Chinese: 30 techniques
Italian: 34 techniques
French/Western European: 82 techniques
Middle Eastern/North African: 78 techniques
East Asian: 82 techniques
Japanese: 82 techniques
